In [1]:
import torch
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
print("Using device:", device)
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
from sklearn.metrics import classification_report
import pandas as pd
import matplotlib.pyplot as plt

Using device: cuda:1


In [2]:
import os
from tqdm import tqdm
os.environ["WANDB_DISABLED"] = "true"

In [3]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [4]:
seed = 25
# random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

In [5]:
train_data = pd.read_csv('datasets/subtask_1/en/train.tsv',sep='\t')
train_data = train_data.reset_index(drop=True)
print(train_data.head())

      id                                               text      label
0  12322  you need to stop the engine and wait until it ...  generated
1   1682  The Commission shall publish the report; an in...  generated
2  22592  I have not been tweeting a lot lately, but I d...  generated
3  17390  I pass my exam and really thankgod for that bu...      human
4  30453  The template will have 3 parts: a mustache sha...      human


In [6]:
from sklearn.model_selection import train_test_split
train_data_texts = train_data['text'].to_list()
train_data_labels = train_data['label'].to_list()
train_data_labels = [0 if x=='human' else 1 for x in train_data_labels]
train_texts, test_texts, train_labels, test_labels = train_test_split(train_data_texts, train_data_labels, test_size=0.1, random_state=25)
train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=0.1, random_state=25)
print('train data size: ', len(train_texts))
print('validation data size: ', len(val_texts))
print('test data size: ', len(test_texts))

train data size:  27414
validation data size:  3046
test data size:  3385


In [7]:
from transformers import AdamW, AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding
from transformers import BertTokenizer, BertModel
from transformers import TrainingArguments, Trainer
bert_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
bert_model = BertModel.from_pretrained("bert-base-uncased").to(device)
print("Model Configurations")
print()
print(bert_model.config)

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Model Configurations

BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.24.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}



In [8]:
def get_bert_embeddings(text):
    # Tokenize input text
    encoded_input = bert_tokenizer(text, padding=True, truncation=True, max_length=512, return_tensors='pt').to(device)
    #get bert embeddings
    with torch.no_grad():
        bert_output = bert_model(**encoded_input)
    bert_embeddings = bert_output.last_hidden_state[:,0,:].cpu().numpy()
    return bert_embeddings

#get train embeddings
train_embeddings = []
for text in tqdm(train_texts):
    train_embeddings.append(get_bert_embeddings(text))
train_embeddings = np.array(train_embeddings)
train_embeddings = np.squeeze(train_embeddings, axis=1)
print('train embeddings shape: ', train_embeddings.shape)

100%|████████████████████████████████████| 27414/27414 [04:28<00:00, 102.00it/s]


train embeddings shape:  (27414, 768)


In [9]:
#get validation embeddings
val_embeddings = []
for text in tqdm(val_texts):
    val_embeddings.append(get_bert_embeddings(text))
val_embeddings = np.array(val_embeddings)
val_embeddings = np.squeeze(val_embeddings, axis=1)
print('validation embeddings shape: ', val_embeddings.shape) #shape: (num_samples, 1, 768)


#get test embeddings
test_embeddings = []
for text in tqdm(test_texts):
    test_embeddings.append(get_bert_embeddings(text))
test_embeddings = np.array(test_embeddings)
test_embeddings = np.squeeze(test_embeddings, axis=1)
print('test embeddings shape: ', test_embeddings.shape) #shape: (num_samples, 1, 768)

100%|██████████████████████████████████████| 3046/3046 [00:29<00:00, 103.02it/s]


validation embeddings shape:  (3046, 768)


100%|██████████████████████████████████████| 3385/3385 [00:32<00:00, 103.20it/s]

test embeddings shape:  (3385, 768)


In [10]:
import string
def count_punctuations(text):
    count = sum([1 for char in text if char in string.punctuation])
    return count

train_punc = []
for text in train_texts:
    train_punc.append(count_punctuations(text))
train_punc = np.array(train_punc)

val_punc = []
for text in val_texts:
    val_punc.append(count_punctuations(text))
val_punc = np.array(val_punc)

test_punc = []
for text in test_texts:
    test_punc.append(count_punctuations(text))
test_punc = np.array(test_punc)
print('train punc shape: ', train_punc.shape) #shape: (num_samples, 1)

train punc shape:  (27414,)


In [11]:
def count_capital_letters(text):
    count = sum([1 for char in text if char.isupper()])
    return count

train_capital = []
for text in train_texts:
    train_capital.append(count_capital_letters(text))
train_capital = np.array(train_capital)

val_capital = []
for text in val_texts:
    val_capital.append(count_capital_letters(text))
val_capital = np.array(val_capital)

test_capital = []
for text in test_texts:
    test_capital.append(count_capital_letters(text))
test_capital = np.array(test_capital)
print('train capital shape: ', train_capital.shape) #shape: (num_samples, 1)

train capital shape:  (27414,)


In [12]:
#function to perform sentiment analysis on a spanish text
from transformers import pipeline
sentiment_analysis = pipeline("sentiment-analysis", model="nlptown/bert-base-multilingual-uncased-sentiment")

In [31]:
def get_sentiment(text):
    sentiment = sentiment_analysis(text)[0]['label']
    #remove stars
    #if its 1 star return 1
    if sentiment == '1 star':
        sentiment = 1
    #if its 2 stars return 2
    elif sentiment == '2 stars':
        sentiment = 2
    #if its 3 stars return 3
    elif sentiment == '3 stars':
        sentiment = 3
    #if its 4 stars return 4
    elif sentiment == '4 stars':
        sentiment = 4
    #if its 5 stars return 5
    elif sentiment == '5 stars':
        sentiment = 5
    return sentiment #dim: (num_samples, 1) range: [1,5]

train_sentiment = []
for text in tqdm(train_texts):
    train_sentiment.append(get_sentiment(text))
train_sentiment = np.array(train_sentiment)

val_sentiment = []
for text in tqdm(val_texts):
    val_sentiment.append(get_sentiment(text))
val_sentiment = np.array(val_sentiment)

test_sentiment = []
for text in tqdm(test_texts):
    test_sentiment.append(get_sentiment(text))
test_sentiment = np.array(test_sentiment)
print('train sentiment shape: ', train_sentiment.shape) #shape: (num_samples, 1)

100%|███████████████████████████████████████| 3385/3385 [02:27<00:00, 22.97it/s]

train sentiment shape:  (27414,)


In [19]:
import spacy
nlp = spacy.load("en_core_web_sm")
#function to get pos tags for each category
def get_pos(text):
    doc = nlp(text)
    adj_count = 0
    noun_count = 0
    verb_count = 0
    adp_count = 0
    det_count = 0
    for token in doc:
        if token.pos_ == 'ADJ':
            adj_count += 1
        elif token.pos_ == 'NOUN':
            noun_count += 1
        elif token.pos_ == 'VERB':
            verb_count += 1
        elif token.pos_ == 'ADP':
            adp_count += 1
        elif token.pos_ == 'DET':
            det_count += 1
    return [adj_count, noun_count, verb_count, adp_count, det_count] #dim: (num_samples, 5) 

train_pos = []
for text in tqdm(train_texts):
    train_pos.append(get_pos(text))
train_pos = np.array(train_pos)

val_pos = []
for text in tqdm(val_texts):
    val_pos.append(get_pos(text))
val_pos = np.array(val_pos)

test_pos = []
for text in tqdm(test_texts):
    test_pos.append(get_pos(text))
test_pos = np.array(test_pos)
print('train pos shape: ', train_pos.shape) #shape: (num_samples, 5)

100%|███████████████████████████████████████| 3385/3385 [00:34<00:00, 99.40it/s]

train pos shape:  (27414, 5)


In [21]:
from transformers import pipeline
ner_analysis = pipeline("ner", model="balamurugan1603/bert-finetuned-ner")

In [22]:
def get_ner(text):
    ner = ner_analysis(text)
    loc_count = 0
    org_count = 0
    per_count = 0
    misc_count = 0
    for item in ner:
        if item['entity'] == 'B-LOC' or item['entity'] == 'I-LOC':
            loc_count += 1
        elif item['entity'] == 'B-ORG' or item['entity'] == 'I-ORG':
            org_count += 1
        elif item['entity'] == 'B-PER' or item['entity'] == 'I-PER':
            per_count += 1
        elif item['entity'] == 'B-MISC' or item['entity'] == 'I-MISC':
            misc_count += 1
    return [loc_count, org_count, per_count, misc_count] #dim: (num_samples, 4)

train_ner = []
for text in tqdm(train_texts):
    train_ner.append(get_ner(text))
train_ner = np.array(train_ner)

val_ner = []
for text in tqdm(val_texts):
    val_ner.append(get_ner(text))
val_ner = np.array(val_ner)

test_ner = []
for text in tqdm(test_texts):
    test_ner.append(get_ner(text))
test_ner = np.array(test_ner)
print('train ner shape: ', train_ner.shape) #shape: (num_samples, 4)

100%|███████████████████████████████████████| 3385/3385 [02:30<00:00, 22.47it/s]

train ner shape:  (27414, 4)


In [23]:
train_punc = train_punc.reshape(-1,1)
val_punc = val_punc.reshape(-1,1)
test_punc = test_punc.reshape(-1,1)

train_capital = train_capital.reshape(-1,1)
val_capital = val_capital.reshape(-1,1)
test_capital = test_capital.reshape(-1,1)

train_sentiment = train_sentiment.reshape(-1,1)
val_sentiment = val_sentiment.reshape(-1,1)
test_sentiment = test_sentiment.reshape(-1,1)

In [33]:
#concatenate all features
train_features = np.concatenate((train_punc, train_capital,train_sentiment, train_pos, train_ner), axis=1) #dim: (num_samples, 11)
val_features = np.concatenate((val_punc, val_capital,val_sentiment, val_pos, val_ner), axis=1) #dim: (num_samples, 11)
test_features = np.concatenate((test_punc, test_capital,test_sentiment, test_pos, test_ner), axis=1) #dim: (num_samples, 11)
print('train features shape: ', train_features.shape) #shape: (num_samples, 11)
print(train_features[0])
#concatenate all features
# train_features = np.concatenate((train_pos, train_ner), axis=1) #dim: (num_samples, 11)
# val_features = np.concatenate((val_pos, val_ner), axis=1) #dim: (num_samples, 11)
# test_features = np.concatenate((test_pos, test_ner), axis=1) #dim: (num_samples, 11)
# print('train features shape: ', train_features.shape) #shape: (num_samples, 11)
# print(train_features[0])

train features shape:  (27414, 12)
[ 7  6  2  7 24 10  7 12  0  0  0  0]


In [34]:
#save these features in a file
np.save('en_train_features.npy', train_features)
np.save('en_val_features.npy', val_features)
np.save('en_test_features.npy', test_features)

In [26]:
# from sklearn.decomposition import PCA
# # Set the number of components you want to keep
# n_components = 15
# # Fit PCA on the validation embeddings and transform them
# pca = PCA(n_components=n_components)

In [27]:
# train_embeddings_pca = pca.fit_transform(train_embeddings)
# print('train embeddings pca shape: ', train_embeddings_pca.shape) #shape: (num_samples, n_components)

# val_embeddings_pca = pca.transform(val_embeddings)
# print('validation embeddings pca shape: ', val_embeddings_pca.shape) #shape: (num_samples, n_components)

# test_embeddings_pca = pca.transform(test_embeddings)
# print('test embeddings pca shape: ', test_embeddings_pca.shape) #shape: (num_samples, n_components)

In [28]:
# from pysentimiento import create_analyzer
# analyzer = create_analyzer(task="sentiment", lang="es")
# text = "Este es un ejemplo de texto con sentimiento."

# result = analyzer.predict(text)

# pos_prob = result.prob_pos
# neg_prob = result.prob_neg
# neu_prob = result.prob_neu

# print("Positive Probability:", pos_prob)
# print("Negative Probability:", neg_prob)
# print("Neutral Probability:", neu_prob)

In [35]:
batch_size = 32
train_dataset = torch.utils.data.TensorDataset(torch.tensor(train_embeddings), torch.tensor(train_labels), torch.tensor(train_features))
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

valid_dataset = torch.utils.data.TensorDataset(torch.tensor(val_embeddings), torch.tensor(val_labels), torch.tensor(val_features))
val_loader = torch.utils.data.DataLoader(dataset=valid_dataset, batch_size=batch_size, shuffle=False)

test_dataset = torch.utils.data.TensorDataset(torch.tensor(test_embeddings), torch.tensor(test_labels), torch.tensor(test_features))
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

for embeddings, labels, f in train_loader:
    print(embeddings.shape)
    print(labels.shape)
    print(f.shape)
    break

torch.Size([32, 768])
torch.Size([32])
torch.Size([32, 12])


In [36]:
# # Define neural network architecture
# import torch.nn as nn
# import torch.nn.functional as F

# #create a neural network to use the embeddings and do classification
# class Net(nn.Module):
#     def __init__(self, input_size, hidden_size, num_classes):
#         super(Net, self).__init__()
#         self.fc1 = nn.Linear(input_size, hidden_size) 
#         self.dropout1 = nn.Dropout(0.1)
#         self.fc2 = nn.Linear(hidden_size, num_classes)  

#     def forward(self, x):
#         # out = F.relu(self.bn1(self.fc1(x)))
#         out = F.relu(self.fc1(x))
#         out = self.dropout1(out)
#         out = self.fc2(out)
#         return out
    
# # Hyperparameters
# input_size = 768
# hidden_size = 128
# num_classes = 2
# num_epochs = 20
# learning_rate = 0.001

In [37]:
# Define neural network architecture
import torch.nn as nn
import torch.nn.functional as F

#create a neural network to use the embeddings and do classification
class Net(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, num_classes):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size1) 
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)
        self.fc3 = nn.Linear(hidden_size2+12, num_classes) #11 is the number of features
        self.dropout = nn.Dropout(0.5)

    def forward(self, x, f):
        out = F.relu(self.fc1(x))
        out = self.dropout(out)
        out = F.relu(self.fc2(out))
        out = torch.cat((out, f), dim=1)    
        out = self.fc3(out)
        return out
    
# Hyperparameters
input_size = 768
hidden_size1 = 256
hidden_size2 = 28
num_classes = 2
num_epochs = 20
learning_rate = 0.001

In [38]:
# Create a model from the neural network
# model = Net(input_size, hidden_size, num_classes).to(device)
model = Net(input_size, hidden_size1, hidden_size2, num_classes).to(device)
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [39]:
# from tqdm import tqdm

# best_val_acc = 0.0
# total_step = len(train_loader)
# half_epoch_step = total_step // 2

# for epoch in range(num_epochs):
#     running_loss = 0.0
#     for i, (embeddings, labels) in tqdm(enumerate(train_loader), total=total_step, desc=f"Epoch {epoch+1}/{num_epochs}", unit="batch"):
#         # Move tensors to the configured device
#         embeddings = embeddings.to(device)
#         labels = labels.to(device)

#         # Forward pass
#         outputs = model(embeddings)
#         loss = criterion(outputs, labels)

#         # Backward and optimize
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()

#         running_loss += loss.item()

#         # Print loss every half epoch
#         if (i+1) % half_epoch_step == 0:
#             avg_loss = running_loss / half_epoch_step
#             print(f"Epoch {epoch+1}/{num_epochs} Loss after {i+1} batches: {avg_loss:.4f}")
#             running_loss = 0.0
            
#     # Validate the model
#     with torch.no_grad():
#         correct = 0
#         total = 0
#         for embeddings, labels in val_loader:
#             embeddings = embeddings.to(device)
#             labels = labels.to(device)
#             outputs = model(embeddings)
#             _, predicted = torch.max(outputs.data, 1)
#             total += labels.size(0)
#             correct += (predicted == labels).sum().item()

#         # Print validation stats
#         val_acc = 100 * correct / total
#         print(f'Epoch {epoch+1}/{num_epochs} Validation Accuracy: {val_acc:.2f} %')

#         # Save the model if the validation accuracy is better than the previous best
#         if val_acc > best_val_acc:
#             best_val_acc = val_acc
#             torch.save(model.state_dict(), 'best_model.pt')
#             print(f'Saved model with validation accuracy: {best_val_acc:.2f} %')

In [41]:
from tqdm import tqdm

best_val_loss = float('inf') # initialize best validation loss to infinity
total_step = len(train_loader)
half_epoch_step = total_step // 2

for epoch in range(num_epochs):
    running_loss = 0.0
    for i, (embeddings, labels, f) in tqdm(enumerate(train_loader), total=total_step, desc=f"Epoch {epoch+1}/{num_epochs}", unit="batch"):
        # Move tensors to the configured device
        embeddings = embeddings.to(device)
        labels = labels.to(device)
        f = f.to(device)
        
        # Forward pass
        outputs = model(embeddings, f)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        # Print loss every half epoch
        if (i+1) % half_epoch_step == 0:
            avg_loss = running_loss / half_epoch_step
            print(f"Epoch {epoch+1}/{num_epochs} Loss after {i+1} batches: {avg_loss:.4f}")
            running_loss = 0.0
            
    # Validate the model
    with torch.no_grad():
        val_loss = 0.0
        correct = 0
        total = 0
        for embeddings, labels, f in val_loader:
            embeddings = embeddings.to(device)
            labels = labels.to(device)
            f = f.to(device)
            outputs = model(embeddings, f)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        # Print validation stats
        val_acc = 100 * correct / total
        val_loss = val_loss / len(val_loader)
        print(f'Epoch {epoch+1}/{num_epochs} Validation Accuracy: {val_acc:.2f} %, Validation Loss: {val_loss:.4f}')

        # Save the model if the validation loss is better than the previous best
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), 'best_model.pt')
            print(f'Saved model with validation loss: {best_val_loss:.4f}')


Epoch 1/20:  57%|██████████████▏          | 486/857 [00:01<00:00, 388.81batch/s]

Epoch 1/20 Loss after 428 batches: 0.2585


Epoch 1/20: 100%|█████████████████████████| 857/857 [00:02<00:00, 377.41batch/s]


Epoch 1/20 Loss after 856 batches: 0.2524
Epoch 1/20 Validation Accuracy: 85.33 %, Validation Loss: 0.3428
Saved model with validation loss: 0.3428


Epoch 2/20:  57%|██████████████▎          | 489/857 [00:01<00:00, 376.52batch/s]

...

Epoch 19/20 Loss after 856 batches: 0.2057
Epoch 19/20 Validation Accuracy: 84.96 %, Validation Loss: 0.3854


Epoch 20/20:  55%|█████████████▎          | 475/857 [00:01<00:00, 389.61batch/s]

Epoch 20/20 Loss after 428 batches: 0.2055


Epoch 20/20: 100%|████████████████████████| 857/857 [00:02<00:00, 389.08batch/s]


Epoch 20/20 Loss after 856 batches: 0.2012
Epoch 20/20 Validation Accuracy: 85.72 %, Validation Loss: 0.3801


In [42]:
with torch.no_grad():
    correct = 0
    total = 0
    predicted_labels = []
    true_labels = []
    for embeddings, labels, f in tqdm(test_loader):
        embeddings = embeddings.to(device)
        labels = labels.to(device)
        f = f.to(device)
        outputs = model(embeddings,f)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        predicted_labels.extend(predicted.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())
    #generate classification report
    test_report = classification_report(true_labels, predicted_labels)

100%|████████████████████████████████████████| 106/106 [00:00<00:00, 801.27it/s]


In [43]:
print(test_report)

              precision    recall  f1-score   support

           0       0.85      0.84      0.85      1703
           1       0.84      0.85      0.85      1682

    accuracy                           0.85      3385
   macro avg       0.85      0.85      0.85      3385
weighted avg       0.85      0.85      0.85      3385

